In [ ]:
from PySide6.QtCore import (QCoreApplication, QDate, QDateTime, QLocale,
    QMetaObject, QObject, QPoint, QRect, QTimer, QTimer,
    QSize, QTime, QUrl, Qt)
from PySide6.QtGui import (QBrush, QColor, QConicalGradient, QCursor,
    QFont, QFontDatabase, QGradient, QIcon, QIntValidator,
    QImage, QKeySequence, QLinearGradient, QPainter,
    QPalette, QPixmap, QRadialGradient, QTransform)
from PySide6.QtWidgets import (QApplication, QComboBox, QFrame, QHBoxLayout,
    QLabel, QLineEdit, QMainWindow, QPushButton, QFileDialog, QDialog,
    QSizePolicy, QWidget)
import sys
import time
import datetime
import json
from pathlib import Path

import cv2
from ui_yolo11 import *
from yolo import *
from rknnlite.api import RKNNLite

TASKS = ["detect", "pose", "seg", "obb"]
DATASETS = ["COCO", "FIRE", "CRACK", "LICENSE", "GARBAGE"]

class MyWindow(QMainWindow, Ui_MainWindow) :
    def __init__(self) :
        super().__init__()
        self.IMG_PATH = None
        self.RK3588_RKNN_MODEL = None
        self.task = TASKS[0]
        self.DATASET = COCO
        self.input_size = 640

        self.setupUi(self)
        self.lineEdit.setValidator(QIntValidator(320, 640, self))

    def model_select(self):
        dialog = QFileDialog(self)
        dialog.setWindowTitle("모델 열기")
        dialog.setNameFilter("Models (*.rknn)")
        dialog.setFileMode(QFileDialog.ExistingFile)
        dialog.setOption(QFileDialog.DontUseNativeDialog, True)  # Qt 다이얼로그 사용
        dialog.resize(400, 300)  # 다이얼로그 크기 축소

        # 커서 위치 (메뉴 클릭 위치)
        cursor_pos = QCursor.pos()

        # 다이얼로그 띄운 뒤 위치 이동
        def position_and_exec():
            dialog.move(cursor_pos)
            dialog.exec()

        QTimer.singleShot(0, position_and_exec)

        # 선택 파일 처리
        def on_file_selected():
            selected_files = dialog.selectedFiles()
            if selected_files:
                file_path = selected_files[0]
                #self.load_image_from_path(file_path)
                print( file_path )
                self.RK3588_RKNN_MODEL = file_path
                file_path = Path( file_path )
                self.pushButton_2.setText( file_path.name )

        dialog.fileSelected.connect(on_file_selected)

    def image_select(self):
        dialog = QFileDialog(self)
        dialog.setWindowTitle("이미지 열기")
        dialog.setNameFilter("Images (*.png *.jpg *.jpeg *.bmp *.gif)")
        dialog.setFileMode(QFileDialog.ExistingFile)
        dialog.setOption(QFileDialog.DontUseNativeDialog, True)  # Qt 다이얼로그 사용
        dialog.resize(400, 300)  # 다이얼로그 크기 축소

        # 커서 위치 (메뉴 클릭 위치)
        cursor_pos = QCursor.pos()

        # 다이얼로그 띄운 뒤 위치 이동
        def position_and_exec():
            dialog.move(cursor_pos)
            dialog.exec()

        QTimer.singleShot(0, position_and_exec)

        # 선택 파일 처리
        def on_file_selected():
            selected_files = dialog.selectedFiles()
            if selected_files:
                file_path = selected_files[0]
                #self.load_image_from_path(file_path)
                print( file_path )
                self.IMG_PATH = file_path
                file_path = Path(file_path)
                self.pushButton_3.setText( file_path.name )
                self.load_image_from_path(self.IMG_PATH)

        dialog.fileSelected.connect(on_file_selected)

    def load_image_from_path(self, file_path):
        image = cv2.imread(file_path)
        self.image, ratio = letterbox(image, 415, 442, 0)
        image_rgb = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        h, w, ch = image_rgb.shape
        bytes_per_line = ch * w
        qimage = QImage(image_rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        self.label_4.setPixmap(pixmap)

    def task_select(self, index):
        #print(self.comboBox.itemText(index), index)
        self.task = TASKS[index]
        print( self.task)

    def dataset_select(self, index):
        #print(self.comboBox_3.itemText(index), index)
        self.DATASET = index
        print( self.DATASET )

    def cvimg_to_qpixmap(self, cv_img):
        image_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        height, width, channel = image_rgb.shape
        bytes_per_line = channel * width
        qimg = QImage(cv_img.data, width, height, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimg)
    
    def inference(self):
        print("infer click!!")
        if self.IMG_PATH is None:
            print("Image not select!!")
        elif self.RK3588_RKNN_MODEL is None:
            print("RKNN Model not select!!")
        else:
            print( self.task)
            print( self.DATASET )
            self.input_size = int(self.lineEdit.text())
            print( self.input_size )
            if self.task == "detect" :
                self.yolo = Yolo( "detect", self.RK3588_RKNN_MODEL, input_size = self.input_size, DATASET=self.DATASET)
                #img_org = cv2.imread(IMG_PATH)
                boxes, classes, scores = self.yolo.detect(self.image)
                if boxes is not None:
                    self.yolo.detect.draw(self.image, boxes, scores, classes, "all", label=True)
                    pixmap = self.cvimg_to_qpixmap(self.image)
                    self.label_4.setPixmap(pixmap)
                    self.lineEdit_7.setText(self.yolo.detect.infertime)
            elif self.task == "pose" :
                self.yolo = Yolo( "pose", self.RK3588_RKNN_MODEL, input_size = self.input_size, DATASET=self.DATASET, CALC_ANGLE=True)
                predbox = self.yolo.pose(self.image)
                self.yolo.pose.draw(self.image, predbox, box_vis = False, angle=False)
                pixmap = self.cvimg_to_qpixmap(self.image)
                self.label_4.setPixmap(pixmap)
                self.lineEdit_7.setText(self.yolo.pose.infertime)
            elif self.task == "seg" :
                self.yolo = Yolo( "seg", self.RK3588_RKNN_MODEL, input_size = self.input_size, DATASET=self.DATASET)
                boxes, classes, scores, seg_img = self.yolo.seg(self.image)
                if boxes is not None:
                    img_p = self.yolo.seg.merge_seg(self.image, seg_img, classes)
                    self.yolo.seg.draw(img_p, boxes, scores, classes, "all", label=True)
                    pixmap = self.cvimg_to_qpixmap(img_p)
                    self.label_4.setPixmap(pixmap)
                    self.lineEdit_7.setText(self.yolo.seg.infertime)
            elif self.task == "obb" :
                self.yolo = Yolo( "obb", self.RK3588_RKNN_MODEL, input_size = self.input_size)
                predbox = self.yolo.obb(self.image)
                draw_img = self.yolo.obb.draw(predbox)
                pixmap = self.cvimg_to_qpixmap(draw_img)
                self.label_4.setPixmap(pixmap)
                self.lineEdit_7.setText(self.yolo.obb.infertime)


    def closeEvent(self, event):
        self.yolo.release()
        event.accept()  # 종료 계속 진행
        
if __name__ == "__main__":
    import sys
    if not QApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QApplication.instance()

    myWindow = MyWindow() 
    myWindow.show()
    sys.exit(app.exec())


obb
/home/darkice/ExtUSB/Work/rknn/rknn_examples/Yolo11_rknn/models/obb/yolo11n-obb-RK3588_i8.rknn
0
/home/darkice/ExtUSB/Work/rknn/rknn_examples/Yolo11_rknn/images/obb.jpg


W rknn-toolkit-lite2 version: 2.3.2
W Query dynamic range failed. Ret code: RKNN_ERR_MODEL_INVALID. (If it is a static shape RKNN model, please ignore the above warning message.)


infer click!!
obb
0
640
YOLO : --> Load YOLO model
done
YOLO : --> Init runtime environment YOLO
I RKNN: [00:44:32.588] RKNN Runtime Information, librknnrt version: 2.3.2 (429f97ae6b@2025-04-09T09:09:27)
I RKNN: [00:44:32.588] RKNN Driver Information, version: 0.9.8
I RKNN: [00:44:32.588] RKNN Model Information, version: 6, toolkit version: 2.3.2(compiler version: 2.3.2 (e045de294f@2025-04-07T19:48:25)), target: RKNPU v2, target platform: rk3588, framework name: ONNX, framework layout: NCHW, model inference type: static_shape
done
W RKNN: [00:44:32.598] query RKNN_QUERY_INPUT_DYNAMIC_RANGE error, rknn model is static shape type, please export rknn with dynamic_shapes
YOLO : --> Running model
YOLO : Inference time: 35.75944900512695ms
YOLO : RKNN OBB Release!!


SystemExit: 0

/home/darkice/miniconda3/envs/pyside6rknn/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
